<a href="https://colab.research.google.com/github/Mohamedsayhii/Library/blob/main/G_VRPSPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from random import choice, randint, randrange, random
from typing import List

In [ ]:
Client = List[int]
Genome = List[int] #solution
clientsNumber = 50
numberOfClientsInCluster = 10
populationSize = 50
weightLimit = 100 #the vehicle capacity
mutationProbability = 0.5

def generateClient() -> Client:
  return [randint(-50, 50), randint(-50, 50), randint(0, 10), randint(-10, 0)] #Each client is the form of a list [x, y, pickup, delivery]

def initializeDatabase(length: int):
  listOfClients = [[0,0,0,0]] #Client zero is the depot
  for _ in range(length):
    listOfClients.append(generateClient())
  return listOfClients

def generateGenome(customers, customersToServePerRoute) -> Genome: #Generate one solution
  routes = []
  visited = []
  j = 0
  numberV = customers / customersToServePerRoute
  while j < numberV:
    i = 0
    route = [0] #Each vehicle have to start from the depot
    while i < customersToServePerRoute:
      randomNumber = randint(1, customers)
      if randomNumber in visited:
        continue
      else:
        route.append(randomNumber)
        visited.append(randomNumber)
        i += 1
    route.append(0) #Each vehicle have to end at the depot
    routes.append(route)
    j += 1
  return routes

def initializePopulation(size):
  return [generateGenome(clientsNumber,numberOfClientsInCluster) for _ in range(size)]

def euclidianDistance(point1, point2):
  return round(sum([(point1[x] - point2[x]) ** 2 for x in range(len(point1))]) ** 0.5, 2)

def evaluateFitness(solution): #calculate the fitness of each solution based on satisfying all the customers demand and the distance traveled by the vehicles
  startingWeight = 50
  distance = 0
  for route in solution:
    for i in range(len(route)-1):
      clientDemand = clients[route[i]-1][2] + clients[route[i]-1][3]
      startingWeight = startingWeight + clientDemand
      if startingWeight < 0 or startingWeight > weightLimit:
        return 10000
      point1 = (clients[route[i]][0], clients[route[i]][1])
      point2 = (clients[route[i+1]][0], clients[route[i+1]][1])
      distance += euclidianDistance(point1, point2)
  return distance

def binaryTournamentSelection(population, fitnesses):
  individual1 = population[randint(0, len(population) - 1)]
  individual2 = population[randint(0, len(population) - 1)]

  individual1Fitness = fitnesses[population.index(individual1)]
  individual2Fitness = fitnesses[population.index(individual2)]

  if individual1Fitness < individual2Fitness:
    return individual1
  else:
    return individual2

def crossover(parent1, parent2):
    size = len(parent1)
    # Choose a random crossover point
    crossover_point = random.randint(0, size-1)
    # Create the offspring
    offspring = parent1[:crossover_point] + parent2[crossover_point:]
    return offspring

def orderedCrossover(parent1, parent2):
    size = len(parent1)
    # Choose two random crossover points
    crossoverPoint1 = randint(0, size-1)
    crossoverPoint2 = randint(0, size-1)
    # Ensure crossover points are in the correct order
    if crossoverPoint1 > crossoverPoint2:
        crossoverPoint1, crossoverPoint2 = crossoverPoint2, crossoverPoint1
    # Create the offspring
    offspring = [None] * size
    # Inherit the section between the crossover points from parent1
    offspring[crossoverPoint1:crossoverPoint2+1] = parent1[crossoverPoint1:crossoverPoint2+1]
    # Fill in the rest of the offspring using the genes from parent2
    p2_index = 0
    for i in range(size):
        if offspring[i] is not None:
            continue
        while parent2[p2_index] in offspring:
            p2_index += 1
        offspring[i] = parent2[p2_index]

    return offspring

def exchangeMutation1(solution, mutationProbability):
    size = len(solution[0])
    i = randint(1, len(solution) - 1)
    j = randint(1, len(solution) - 1)
    # Apply mutation with a probability of mutation_rate
    #if random() < mutationProbability:
        # Choose two random positions to exchange
    pos1 = randint(0, size-1)
    pos2 = randint(0, size-1)
        # Exchange the genes at the chosen positions
    solution[i][pos1], solution[j][pos2] = solution[j][pos2], solution[i][pos1]
    return solution

def exchangeMutation2(solution, mutationProbability):
    size = len(solution[0])
    # Apply mutation with a probability of mutation_rate
    if random() < mutationProbability:
      for i in range(len(solution) - 1):
        # Choose two random positions to exchange
        pos1 = randint(0, size-1)
        pos2 = randint(0, size-1)
        # Exchange the genes at the chosen positions
        solution[i][pos1], solution[i][pos2] = solution[i][pos2], solution[i][pos1]
    return solution

In [ ]:
def geneticAlgorithm(size, maxAlpha, maxBeta, R):
  population = initializePopulation(size)
  fitnesses = [evaluateFitness(solution) for solution in population]
  alpha, beta, restarts = 0, 0, 0
  while restarts < R:
    print('generation number', restarts + 1)
    while alpha < maxAlpha and beta < maxBeta:
      bestChild = population[0]
      parent1 = binaryTournamentSelection(population, fitnesses)
      parent2 = binaryTournamentSelection(population, fitnesses)
      child1 = orderedCrossover(parent1, parent2)
      child2 = orderedCrossover(parent1, parent2)
      childrenList = [child1, child2]
      randomChild = choice(childrenList)
      if random() < mutationProbability:
        exchangeMutation1(randomChild, 1)
        k = randint(size/2, size - 1)
        if evaluateFitness(randomChild) not in population and evaluateFitness(randomChild) < evaluateFitness(population[k]):
          population[k] = randomChild
          fitnesses[k] = evaluateFitness(randomChild)
        alpha += 1
        print('alpha is:', alpha)
      population.sort()
      if bestChild == population[0]:
        beta += 1
        print('beta is:', beta)
    restarts += 1
  for i in range(len(population) - 1):
    print('solution number', i + 1, 'has a fitness of', fitnesses[i], 'and is:', population[i])


In [ ]:
clients = initializeDatabase(clientsNumber)
geneticAlgorithm(populationSize, 300, 3000, 10)

generation number 1
beta is: 1
beta is: 2
alpha is: 1
beta is: 3
beta is: 4
beta is: 5
alpha is: 2
beta is: 6
alpha is: 3
beta is: 7
alpha is: 4
beta is: 8
alpha is: 5
beta is: 9
alpha is: 6
beta is: 10
alpha is: 7
beta is: 11
beta is: 12
beta is: 13
alpha is: 8
beta is: 14
alpha is: 9
beta is: 15
alpha is: 10
beta is: 16
alpha is: 11
beta is: 17
beta is: 18
alpha is: 12
beta is: 19
beta is: 20
alpha is: 13
beta is: 21
alpha is: 14
beta is: 22
beta is: 23
alpha is: 15
beta is: 24
alpha is: 16
beta is: 25
alpha is: 17
beta is: 26
alpha is: 18
beta is: 27
alpha is: 19
beta is: 28
beta is: 29
beta is: 30
alpha is: 20
beta is: 31
beta is: 32
beta is: 33
beta is: 34
alpha is: 21
beta is: 35
alpha is: 22
beta is: 36
alpha is: 23
beta is: 37
alpha is: 24
beta is: 38
beta is: 39
alpha is: 25
beta is: 40
beta is: 41
alpha is: 26
beta is: 42
beta is: 43
beta is: 44
alpha is: 27
beta is: 45
beta is: 46
alpha is: 28
beta is: 47
alpha is: 29
beta is: 48
beta is: 49
alpha is: 30
beta is: 50
alpha is